In [16]:
# Import Library

import pickle
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime

warnings.filterwarnings('ignore')

In [2]:
# Read the Files

with open('m2/imputer_num.pkl', 'rb') as file_1:
    imputer_num=pickle.load(file_1)
    
with open('m2/imputer_cat1.pkl', 'rb') as file_2:
    imputer_cat1=pickle.load(file_2)

with open('m2/imputer_cat2.pkl', 'rb') as file_3:
    imputer_cat2=pickle.load(file_3)
    
with open('m2/windsoriser.pkl', 'rb') as file_4:
    windsoriser=pickle.load(file_4)
    
with open('m2/Stdscaler.pkl', 'rb') as file_8:
    Stdscaler=pickle.load(file_8)
    
with open('m2/MMscaler.pkl', 'rb') as file_8:
    MMscaler=pickle.load(file_8)

with open('m2/RBscaler.pkl', 'rb') as file_8:
    RBscaler=pickle.load(file_8)

with open('m2/ohe_pipeline.pkl', 'rb') as file_8:
    ohe_pipeline=pickle.load(file_8)
    
with open('m2/ode_pipeline.pkl', 'rb') as file_8:
    ode_pipeline=pickle.load(file_8)

with open('m2/preprocessor.pkl', 'rb') as file_9:
    preprocessor=pickle.load(file_9)  
    
with open('m2/logreg_pipe.pkl', 'rb') as file_9:
    logreg_pipe=pickle.load(file_9)  
    

with open('m2/num_cols.txt', 'r') as file_5:
    num_cols=json.load(file_5)

with open('m2/nom_cat_cols.txt','r') as file_6: 
    nom_cat_cols=json.load(file_6)
    
with open('m2/ord_cat_cols.txt', 'r') as file_7:
    ord_cat_cols=json.load(file_7) 
    
with open('m2/num_cols_norm.txt', 'r') as file_5:
    num_cols_norm=json.load(file_5)

with open('m2/num_cols_skew.txt','r') as file_6: 
    num_cols_skew=json.load(file_6)
    
with open('m2/num_cols_rob.txt', 'r') as file_7:
    num_cols_rob=json.load(file_7)
    


In [40]:
# Data loading
df=pd.read_csv('m2/data_inf.csv')
df.pop(df.columns.values[0])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 16 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        1 non-null      int64 
 1   job        1 non-null      object
 2   marital    1 non-null      object
 3   education  1 non-null      object
 4   default    1 non-null      object
 5   balance    1 non-null      int64 
 6   housing    1 non-null      object
 7   loan       1 non-null      object
 8   contact    1 non-null      object
 9   day        1 non-null      int64 
 10  month      1 non-null      object
 11  duration   1 non-null      int64 
 12  campaign   1 non-null      int64 
 13  pdays      1 non-null      int64 
 14  previous   1 non-null      int64 
 15  poutcome   1 non-null      object
dtypes: int64(7), object(9)
memory usage: 256.0+ bytes


In [41]:
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,55,retired,married,secondary,no,2312,no,no,unknown,12,jun,60,1,-1,0,unknown


In [42]:
# categorize the education by the level of education
job_cat=[]
for i in range (df.shape[0]):    
    if ((df['job'][i] == "blue-collar") | (df['job'][i] == "technician")):
        job_cat.append('blue_collar')
    elif ((df['job'][i] == "management") | (df['job'][i] == "admin.")):
        job_cat.append('white_collar')
    elif ((df['job'][i] == 'unemployed')| (df['job'][i] == 'student')):
        job_cat.append("unemployed")
    else:
        job_cat.append(df['job'][i])
df['job_cat']=job_cat
print ("Job Category :",df['job_cat'].unique())

Job Category : ['retired']


In [43]:
# Labelling the month into month_num
df['month_num'] = df['month'].apply(lambda x: datetime.strptime(x, '%b').month)

In [44]:
# categorize the education by the level of education
active_month=[]
for i in range (df.shape[0]):    
    if (df['month_num'][i] in [1,2,4,6,7,8]):
        active_month.append('active')
    elif df['month_num'][i]==5:
        active_month.append('most_active')
    else:
        active_month.append('less_active')
df['active_month']=active_month
print ("Month Categpry :",df['active_month'].unique())
print ("Count of Unique Values :",df['active_month'].nunique())

Month Categpry : ['active']
Count of Unique Values : 1


In [45]:
# Feature Selection
X_test = df.drop(['job', 'month', 'contact', 'poutcome'], axis=1)
X_test

,age,marital,education,default,balance,housing,loan,day,duration,campaign,pdays,previous,job_cat,month_num,active_month
0,55,married,secondary,no,2312,no,no,12,60,1,-1,0,retired,6,active


In [46]:
# Split the X based on the column type
X_test_num = X_test[num_cols]
X_test_cat_nom = X_test[nom_cat_cols]
X_test_cat_ord = X_test[ord_cat_cols]

In [47]:
# Impute missing value in test data
# Index ter-reset
X_test_num=pd.DataFrame(imputer_num.transform(X_test_num), columns=X_test_num.columns.values)
X_test_cat_nom=pd.DataFrame(imputer_cat1.transform(X_test_cat_nom), columns=X_test_cat_nom.columns.values)
X_test_cat_ord=pd.DataFrame(imputer_cat2.transform(X_test_cat_ord), columns=X_test_cat_ord.columns.values)

In [48]:
# Handling Outlier
X_test_num_capped = windsoriser.transform(X_test_num)

In [49]:
# Define the numerical columns
X_test_num_norm = X_test_num_capped[num_cols_norm]
X_test_num_skew = X_test_num_capped[num_cols_skew]
X_test_num_rob = X_test_num_capped[num_cols_rob]

In [50]:
#Transform the numerical test data
X_test_num_norm_scaled = pd.DataFrame(Stdscaler.transform(X_test_num_norm), columns=num_cols_norm)
X_test_num_skew_scaled = pd.DataFrame(MMscaler.transform(X_test_num_skew), columns=num_cols_skew)
X_test_num_rob_scaled = pd.DataFrame(RBscaler.transform(X_test_num_rob), columns=num_cols_rob)

In [51]:
# Concate the imbalance X data
X_test_imbalanced=pd.concat([X_test_num_norm_scaled,X_test_num_skew_scaled,X_test_num_rob_scaled,
                             X_test_cat_nom, X_test_cat_ord], axis=1)
X_test_imbalanced.isnull().mean()

day             0.0
age             0.0
balance         0.0
duration        0.0
campaign        0.0
pdays           0.0
previous        0.0
marital         0.0
job_cat         0.0
education       0.0
default         0.0
housing         0.0
loan            0.0
active_month    0.0
dtype: float64

In [52]:
# Predict the target
y_pred_test_logreg_pipe = logreg_pipe.predict(X_test_imbalanced)
y_pred_test_logreg_pipe

array([0], dtype=int64)

Berdasarkan model inference, diprediksikan di data baru ini, nasabah tidak berlangganan deposito sehingga tidak akan dijadikan target telemarketing.